# Загрузка библиотек

In [ ]:
!pip install optuna -q
!pip install natasha -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 38.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import pandas as pd
import numpy as np

# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb
from lightgbm import LGBMRegressor, early_stopping, Dataset, LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

import optuna
# from optuna.integration import CatBoostPruningCallback

import seaborn as sns
import datetime
from wordcloud import WordCloud, STOPWORDS

import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")

# sns.set(style="darkgrid")
# %matplotlib inline

import warnings
warnings.simplefilter('ignore')

from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

import gensim.downloader
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.fasttext import FastText
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger

# import mlxtend
# from mlxtend.evaluate import paired_ttest_kfold_cv

# from plotly.offline import iplot
# import cufflinks as cf
# cf.go_offline()
# cf.set_config_file(offline=False, world_readable=True)

# treat all python warnings as lower-level "ignore" events
# warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


фонтанка, лента, риа, дзен?

In [ ]:
df_test = pd.read_csv('/content/test_news.csv')

In [ ]:
df_test

,content
0,Фото: «Фонтанка.ру»ПоделитьсяЭкс-министру обор...
1,В начале февраля 2023 года в Пушкинском районе...
2,Фото: Andy Bao / Getty Images Анастасия Борисо...
3,"Если вы хотели, но так и не съездили на море л..."
4,Сергей Пиняев Фото: Алексей Филиппов / РИА Нов...
...,...
26270,Фото: РИА Новости Алевтина Запольская Главное ...
26271,Вадим Гутцайт Фото: Sergei CHUZAVKOV / Europea...
26272,Фото: Олег Харсеев / Коммерсантъ Александр Кур...
26273,Владимир Зеленский Фото: Yves Herman / Reuters...


In [ ]:
df_test.iloc[18773].content

'Фото: Юрий Мартьянов / «Коммерсантъ» Платон Щукин В Deloitte объяснили причины нежелания российских компаний выходить на IPO, размещая акции на бирже. По словам топ-менеджера Deloitte Екатерины Трофимовой, во многом это связано с тем, что фирмы не хотят много о себе рассказывать и принимать на себя риски, связанные с публичностью. Об этом она рассказала РБК. «Многие ищут недостижимого компромисса: давайте мы на рынок выйдем, но ничего не расскажем и, если что, вообще быстро уйдем с рынка. Закрытость, опасения, что публичность может поставить их в уязвимое положение, связанные прямые и косвенные расходы — все это среди прочего сдерживает приток IPO», — объяснила она. Как стало известно ранее, в российском правительстве задумались об увеличении числа IPO. В числе предложений, которые прорабатывало Минэкономразвития, — освобождение от налогов розничных инвесторов, которые захотят поучаствовать в IPO, и помощь компаниям, которые решат пойти на биржу.'

In [ ]:
df_test.iloc[26270].content

'Фото: РИА Новости Алевтина Запольская Главное управление разведки (ГУР) Министерства обороны Украины подтвердило причастность к убийству депутата Народного совета (НС) и бывшего начальника управления Народной милиции ЛНР полковника Михаила Филипоненко. Об этом говорится в сообщении ГУР в Telegram. 8 ноября стало известно, что в Луганске взорвали автомобиль, в котором находился Филипоненко. Теракт был совершен у дома полковника. По версии ГУР, Филипоненко якобы был причастен к организации пыточных на территории ЛНР. «Специальная операция по ликвидации палача Филипоненко была реализована совместно с представителями движения сопротивления», — говорится в сообщении ведомства. Ранее глава ГУР Кирилл Буданов подтвердил причастность украинской разведки к подрывам медийных лиц в России. По данным издания The Washington Post, ведомство организовало убийство российской журналистки Дарьи Дугиной и военкора Владлена Татарского.'

# Загрузка данных

In [ ]:
# настроим отображение колонок
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# общий датасет собирался из трёх. Можно запустить эту часть кода и уже работать с разделом modeling
# либо собрать датасет заново (разделы ниже)
# тогда потребуется работать с новым собранным датасетом и изменить путь path_train

class Paths:
    path_train = '/content/drive/MyDrive/competitions/NLP2.0/data/parsing_data/df_4_corr.csv'
    path_test = '/content/drive/MyDrive/competitions/NLP2.0/data/test_news.csv'
    path_sub = '/content/drive/MyDrive/competitions/NLP2.0/data/base_submission_news.csv'

paths = Paths()

# Modeling

## Уменьшение размера

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        # else:
        #     df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df


print('train')
train = import_data(paths.path_train)

print('-' * 80)
print('test')
test = import_data(paths.path_test)

train
Memory usage of dataframe is 0.92 MB
Memory usage after optimization is: 0.57 MB
Decreased by 37.5%
--------------------------------------------------------------------------------
test
Memory usage of dataframe is 0.20 MB
Memory usage after optimization is: 0.20 MB
Decreased by 0.0%


## Фильтрация фич

In [ ]:
cat_features = ['content']
text_features = ['content']
cat_features_indxs = [1]
targets = ['topic']

filtered_features = [i for i in train.columns if (i not in targets)]

In [ ]:
# Удаляем дубли
train.drop_duplicates(inplace=True)
# Удаляем пропуски
train.dropna(inplace=True)

print(f'train shape = {train.shape},  test shape = {test.shape}')
print(f'train, test is null: {train.isna().any().any(), test.isna().any().any()}')

train shape = (59001, 2),  test shape = (26275, 1)
train, test is null: (False, False)


## Разделение на фичи и признаки

In [ ]:
# X = train[filtered_features].drop(targets, axis=1, errors="ignore")
# y = train["topic"]

In [ ]:
# X = X[: 1000]
# y = y[: 1000]

In [ ]:
# X.shape, y.shape

## Фиксируем значения

In [ ]:
RND_STATE = 7575
N_EST = 1000
sampler = optuna.samplers.TPESampler(seed=RND_STATE)

In [ ]:
# with open(f'model_{0}.pkl', "wb") as f:
#         pickle.dump(clf_1, f)

In [ ]:
# from optuna.integration import CatBoostPruningCallback, LightGBMPruningCallback, XGBoostPruningCallback
# from optuna.integration.pytorch_lightning import PyTorchLightningPruningCallback
# # optuna.logging.set_verbosity(optuna.logging.WARNING)
# # optuna.logging.set_verbosity(optuna.logging.ERROR)

# Предобработка текстов.

In [ ]:
# X = train[filtered_features].drop(targets, axis=1, errors="ignore")
# y = train["topic"]

In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

In [ ]:
stop_words = stopwords.words('russian')
stop_words.extend(['что', 'это', 'так',
                    'вот', 'быть', 'как',
                    'в', '—', 'к', 'за', 'из', 'из-за',
                    'на', 'ок', 'кстати',
                    'который', 'мочь', 'весь',
                    'еще', 'также', 'свой',
                    'ещё', 'самый', 'ул', 'комментарий',
                    'английский', 'язык'])

Функция для обработки текстов.

In [ ]:
def text_prep(text) -> str:
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)

    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    lemmas = [_.lemma for _ in doc.tokens]
    words = [lemma for lemma in lemmas if lemma.isalpha() and len(lemma) > 2]
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

Обработаем тексты обучения и теста.

In [ ]:
%%time
train['content_clean'] = train.content.apply(text_prep)

CPU times: user 32min 7s, sys: 19min 9s, total: 51min 16s
Wall time: 31min 25s


In [ ]:
%%time
test['content_clean'] = test.content.apply(text_prep)

CPU times: user 32min 58s, sys: 19min 10s, total: 52min 8s
Wall time: 32min 15s


In [ ]:
# train.to_csv('train_prep_2.csv', index=False)
# test.to_csv('test_prep.csv', index=False)

In [ ]:
train.head()

,content,topic,content_clean
0,"Поклонники Гарри Поттера со всего мира смотрят спецэпизод, приуроченный к 20-летию самой волшебной франшизы. 1 январ...",0,поклонник гарри поттер мир смотреть спецэпизод приурочить волшебный франшиза январь документальный фильм секрет созд...
1,Египетский курорт засыпало градом 1 января. В некоторых районах он был крупным и причинял прохожим боль. Фото и виде...,0,египетский курорт засыпало град январь некоторый район крупный причинять прохожий боль фото видео курорт опубликоват...
2,Главное управление по Краснодарскому краю МЧС России 1 января выпустило экстренное предупреждение по лавиноопасности...,0,главный управление краснодарский край мчс россия январь выпустить экстренный предупреждение лавиноопасность информац...
3,В новогоднюю ночь в родильных домах Петербурга и акушерских отделениях городских больниц на свет появились 34 младен...,0,новогодний ночь родильный дом петербург акушерский отделение городской больница свет появиться младенец сообщить смо...
4,"Договор с названием RCEP (Regional Comprehensive Economic Partnership, или Всестороннее региональное экономическое п...",0,договор название rcep regional comprehensive economic partnership всесторонний региональный экономический партнерств...


## Загрузка

Сравнение изначального текста и текста после предобработки.

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/train_prep_3_with_ria.csv')
test = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/data/prep_data/test_prep.csv', encoding='utf-8')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.content_clean.str.split(),
                                                    df.topic,#.values,
                                                    test_size=0.2,
                                                    random_state=RND_STATE)

In [ ]:
X_train.head()

3154     [кремль, видеть, кадр, пожар, якобы, крейсер, ...
7534     [указ, введение, режим, повысить, готовность, ...
19433    [фото, иван, кальминг, коммерсантъ, украинский...
56812    [кадр, shot, зураб, дарахвелидзе, сеть, появит...
56075    [фото, maksim, konstantinov, globallookpress, ...
Name: content_clean, dtype: object

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/competitions/NLP2.0/tmp_data/train_prep_2.csv')
    # path_testw2v = '/content/drive/MyDrive/competitions/NLP2.0/tmp_data/test_prep.csv'

In [ ]:
df.head()

,content,topic,content_clean
0,Поклонники Гарри Поттера со всего мира смотрят...,0,поклонник гарри поттер мир смотреть спецэпизод...
1,Египетский курорт засыпало градом 1 января. В ...,0,египетский курорт засыпало град январь некотор...
2,Главное управление по Краснодарскому краю МЧС ...,0,главный управление краснодарский край мчс росс...
3,В новогоднюю ночь в родильных домах Петербурга...,0,новогодний ночь родильный дом петербург акушер...
4,Договор с названием RCEP (Regional Comprehensi...,0,договор название rcep regional comprehensive e...


In [ ]:
# train.sample(1)[['content', 'content_clean']].values

In [ ]:
# train.head()

# Simple 3 pipe

Изначально думал сразу в один пайплайн поставить и обработку текста и подготовку и обучение. Но решил отказаться от этой идеи из-за времени обучения.

In [ ]:
# class SelectColumnsNormalizeTransformer():
#     def __init__(self, columns=None):
#         self.columns = columns

#     def fit(self, X, y=None, **fit_params):
#         return self

#     def transform(self, X, **transform_params):
#         doc = Doc(X[self.columns])
#         doc.segment(segmenter)
#         doc.tag_morph(morph_tagger)

#         for token in doc.tokens:
#             token.lemmatize(morph_vocab)

#         lemmas = [_.lemma for _ in doc.tokens]
#         words = [lemma for lemma in lemmas if lemma.isalpha() and len(lemma) > 2]
#         filtered_words = [word for word in words if word not in stop_words]
#         return " ".join(filtered_words)
#         # return corpus

# pipe_tf_idf = Pipeline([
#     ('col_selector', SelectColumnsNormalizeTransformer('content')),
#     ('tfidf', TfidfVectorizer())
#     ])

In [ ]:
# from sklearn.pipeline import Pipeline
# from mlxtend.feature_selection import ColumnSelector
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import BernoulliNB





# Bernoulli_pipe = Pipeline([
#     ('col_selector', ColumnSelector(cols=('text'),drop_axis=True)),
#     ('tfidf', TfidfVectorizer()),
#     ('scaler', StandardScaler()),
#     ('bernoulli', BernoulliNB()),
# ])


# Bernoulli_pipe.fit(X_, y_le)

# Simple models

In [ ]:
# Для получения вектора предложения используем tf_idf
class TfidfEmbeddingVectorizer(object):
    """Get tfidf weighted vectors"""
    def __init__(self, model):
        self.word2vec = model.wv
        self.word2weight = None
        self.dim = model.vector_size

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec.get_vector(w) * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])

### Word2Vec



In [ ]:
X_train.head()

21398    [фото, максим, блинов, риа, новость, дарья, ры...
25991    [фото, алексей, сухоруков, риа, новость, анаст...
10146    [фото, евгений, бият, риа, новость, главный, в...
30966    [фото, thomas, lohnes, getty, images, алевтина...
2335     [март, год, банк, провести, прямой, включение,...
Name: content_clean, dtype: object

In [ ]:
X_train.shape

(25839,)

In [ ]:
model_w2v = Word2Vec(sentences=X_train,
                 vector_size=300,
                 min_count=5,
                 window=25,
                 seed=RND_STATE)

### FastText

In [ ]:
model_fast = FastText(sentences=X_train,
                      vector_size=300,
                      min_count=10,
                      window=10,
                      seed=RND_STATE)

## Логистическая регрессия

### W2V

In [ ]:
%%time
pipe = Pipeline([('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=10000,
                                            # class_weight='balanced'
                                            ))])

pipe.fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.83      0.88      0.85      2869
           1       0.81      0.80      0.80       837
           2       0.85      0.73      0.78       400
           3       0.84      0.80      0.82       707
           4       0.98      0.95      0.96       617
           5       0.48      0.44      0.46        27
           6       0.95      0.94      0.94       190
           7       0.85      0.81      0.83       347
           8       0.86      0.80      0.83       466

    accuracy                           0.85      6460
   macro avg       0.83      0.79      0.81      6460
weighted avg       0.85      0.85      0.85      6460

CPU times: user 2min 56s, sys: 18.8 s, total: 3min 15s
Wall time: 2min 25s


### FastText


In [ ]:
%%time
pipe = Pipeline([('fast', TfidfEmbeddingVectorizer(model_fast)),
                 ('scaler', StandardScaler()),
                 ('clf', LogisticRegression(random_state=RND_STATE,
                                            max_iter=10000,
                                            # class_weight='balanced'
                                            ))])

pipe.fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.87      0.84      2869
           1       0.79      0.79      0.79       837
           2       0.82      0.71      0.77       400
           3       0.81      0.75      0.78       707
           4       0.97      0.95      0.96       617
           5       0.60      0.44      0.51        27
           6       0.91      0.92      0.91       190
           7       0.86      0.78      0.82       347
           8       0.84      0.77      0.80       466

    accuracy                           0.83      6460
   macro avg       0.82      0.77      0.80      6460
weighted avg       0.83      0.83      0.83      6460

CPU times: user 3min 5s, sys: 17.9 s, total: 3min 23s
Wall time: 2min 31s


### Optuna для лог_рег

In [ ]:
def fit_log_reg(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {"dual": False,
             "C": trial.suggest_float("C", 2, 4, step=0.1),
             "solver": trial.suggest_categorical("solver",
                                                 ["sag",
                                                  #"saga",
                                                  "lbfgs"
                                                  ]),
             "random_state": RND_STATE,
             "max_iter": trial.suggest_int("max_iter", 10, 10000, step=1000),
             'multi_class': trial.suggest_categorical("multi_class",
                                                      ['auto',
                                                       'ovr',
                                                       'multinomial']),
             'n_jobs': -1,
             'verbose': 0
    }

    # print(f'param["solver"] = {param["solver"]}')
    if param["solver"] == "saga":
        param["penalty"] = trial.suggest_categorical("penalty",
                                             ['l1', 'l2', #'elasticnet',
                                              None])
    else:
         param["penalty"] = trial.suggest_categorical("penalty",
                                             ['l2',
                                              None
                                              ])

    clf = LogisticRegression(**param)

    pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(model_w2v)),
                    ('scaler', StandardScaler()),
                    ('clf', clf)])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_val)
    return pipe, y_pred

In [ ]:
def objective_log(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

    scores_simple_acc, scores_balanced_acc, models = [], [], []

    for train_idx, valid_idx in kf.split(X_train.index):
        train_data = X_train.iloc[train_idx], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_log_reg(trial, train_data, valid_data)

        # scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
        scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break

    result = np.mean(scores_balanced_acc)
    # print(f'result = {result}')

    if return_models:
        return result, models
    else:
        return result

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            sampler=sampler,
                            study_name="Log_Reg_clf")
study.optimize(objective_log,
               n_trials=100,
               n_jobs=-1,
               show_progress_bar=True)

[I 2023-12-31 14:08:43,601] A new study created in memory with name: Log_Reg_clf


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2023-12-31 14:21:28,484] Trial 1 finished with value: 0.8200660071804712 and parameters: {'C': 2.7, 'solver': 'lbfgs', 'max_iter': 5010, 'multi_class': 'multinomial', 'penalty': None}. Best is trial 1 with value: 0.8200660071804712.
[I 2023-12-31 14:23:18,599] Trial 2 finished with value: 0.8592517819569019 and parameters: {'C': 2.1, 'solver': 'sag', 'max_iter': 10, 'multi_class': 'auto', 'penalty': 'l2'}. Best is trial 2 with value: 0.8592517819569019.
[I 2023-12-31 14:24:00,606] Trial 0 finished with value: 0.8125303699931606 and parameters: {'C': 2.3, 'solver': 'lbfgs', 'max_iter': 9010, 'multi_class': 'ovr', 'penalty': None}. Best is trial 2 with value: 0.8592517819569019.
[I 2023-12-31 14:29:48,584] Trial 4 finished with value: 0.8234176569699679 and parameters: {'C': 3.7, 'solver': 'lbfgs', 'max_iter': 4010, 'multi_class': 'multinomial', 'penalty': None}. Best is trial 2 with value: 0.8592517819569019.
[I 2023-12-31 14:32:44,731] Trial 5 finished with value: 0.8686307533772099

In [ ]:
params = ["C",
          'solver',
          'max_iter',
          'multi_class',
          'penalty'
          ]

optuna.visualization.plot_slice(study, params=params,# target_name='accuracy'
)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


Number of finished trials: 100
Best trial:
  Value: 0.872962491801824
  Params: 
    C: 2.7
    solver: lbfgs
    max_iter: 6010
    multi_class: ovr
    penalty: l2


## SVM

# Greid for model

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# parameters = {'model_w2v__min_count':[5, 10, 15, 20], 'model_w2v__window':[5, 10, 15]}
# pipe = Pipeline([('model_w2v', Word2Vec(sentences=X_train,
#                  vector_size=300,
#                 #  min_count=8,
#                 #  window=15,
#                  seed=RND_STATE)),
#                   ('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
#                   ('scaler', StandardScaler()),
#                   ('clf', SVC(random_state=RND_STATE,
#                              max_iter=10000,
#                             #  class_weight='balanced'
#                              ))])
# clf = GridSearchCV(pipe, parameters)

# clf.fit(X_train, y_train)

In [ ]:
model_w2v = Word2Vec(sentences=X_train,
                 vector_size=300,
                 min_count=8,
                 window=15,
                 seed=RND_STATE)

In [ ]:
%%time
pipe = Pipeline([('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                  ('scaler', StandardScaler()),
                  ('clf', SVC(random_state=RND_STATE,
                             max_iter=10000,
                            #  class_weight='balanced'
                             ))])

pipe.fit(X_train, y_train)

### W2V

In [ ]:
%%time
pipe = Pipeline([('w2v_tfidf', TfidfEmbeddingVectorizer(model_w2v)),
                  ('scaler', StandardScaler()),
                  ('clf', SVC(random_state=RND_STATE,
                             max_iter=10000,
                            #  class_weight='balanced'
                             ))])

pipe.fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.89      0.88      2869
           1       0.83      0.85      0.84       837
           2       0.85      0.81      0.83       400
           3       0.85      0.82      0.84       707
           4       0.97      0.96      0.97       617
           5       0.69      0.33      0.45        27
           6       0.96      0.95      0.96       190
           7       0.89      0.89      0.89       347
           8       0.90      0.83      0.86       466

    accuracy                           0.87      6460
   macro avg       0.87      0.82      0.83      6460
weighted avg       0.87      0.87      0.87      6460

CPU times: user 2min 51s, sys: 503 ms, total: 2min 52s
Wall time: 2min 53s


### FastText

In [ ]:
pipe = Pipeline([('fast', TfidfEmbeddingVectorizer(model_fast)),
                 ('scaler', StandardScaler()),
                 ('clf', SVC(random_state=RND_STATE,
                               max_iter=10000,
                               class_weight='balanced'
                               ))
                 ])

pipe.fit(X_train, y_train)

print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.95      0.63      0.76      2213
           1       0.63      0.86      0.73       392
           2       0.51      0.94      0.67       190
           3       0.61      0.90      0.73       300
           4       0.93      0.96      0.94       442
           5       0.22      0.65      0.33        23
           6       0.92      0.95      0.94       202
           7       0.54      0.87      0.67       191
           8       0.59      0.85      0.70       275

    accuracy                           0.76      4228
   macro avg       0.66      0.85      0.72      4228
weighted avg       0.83      0.76      0.77      4228



### Optuna для svm

In [ ]:
def fit_svm(trial, train, val):
    X_train, y_train = train
    X_val, y_val = val

    param = {"C": trial.suggest_float("C", 1, 3, step=0.5),
             "kernel": trial.suggest_categorical("kernel",
                                                 ["linear",
                                                  "poly",
                                                  "rbf",
                                                  'sigmoid'
                                                  ]),
             "random_state": RND_STATE,
             "degree": trial.suggest_int("degree", 2, 5, step=1),
             "max_iter": trial.suggest_int("max_iter", 10, 10000, step=1000),
             'decision_function_shape': trial.suggest_categorical("decision_function_shape",
                                                      ['ovo',
                                                       'ovr']),
             'verbose': 0
    }

    clf = SVC(**param)

    pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(model_w2v)),
                    ('scaler', StandardScaler()),
                    ('clf', clf)])

    pipe.fit(X_train, y_train)

    y_pred = pipe.predict(X_val)
    return pipe, y_pred

In [ ]:
def objective_svm(trial, return_models=False):
    n_splits = 3
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RND_STATE)

    scores_simple_acc, scores_balanced_acc, models = [], [], []

    for train_idx, valid_idx in kf.split(X_train.index):
        train_data = X_train.iloc[train_idx], y_train.iloc[train_idx]
        valid_data = X_train.iloc[valid_idx], y_train.iloc[valid_idx]

        # Подаем trials для перебора
        model, y_pred = fit_svm(trial, train_data, valid_data)

        # scores_simple_acc.append(accuracy_score(y_pred, valid_data[1]))
        scores_balanced_acc.append(balanced_accuracy_score(y_pred, valid_data[1]))
        models.append(model)
        break

    result = np.mean(scores_balanced_acc)

    if return_models:
        return result, models
    else:
        return result

In [ ]:
study = optuna.create_study(direction="maximize",
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),
                            sampler=sampler,
                            study_name="SVM_clf")
study.optimize(objective_svm,
               n_trials=4,
               n_jobs=-1,
               show_progress_bar=True)

[I 2023-12-30 20:19:39,509] A new study created in memory with name: SVM_clf


  0%|          | 0/4 [00:00<?, ?it/s]

[I 2023-12-30 20:23:23,447] Trial 0 finished with value: 0.8736328727865764 and parameters: {'C': 2.0, 'kernel': 'rbf', 'degree': 2, 'max_iter': 2010, 'decision_function_shape': 'ovo'}. Best is trial 0 with value: 0.8736328727865764.
[I 2023-12-30 20:23:30,895] Trial 1 finished with value: 0.6029532277687374 and parameters: {'C': 2.0, 'kernel': 'linear', 'degree': 5, 'max_iter': 9010, 'decision_function_shape': 'ovr'}. Best is trial 0 with value: 0.8736328727865764.
[I 2023-12-30 20:26:35,384] Trial 2 finished with value: 0.6247403749498791 and parameters: {'C': 1.0, 'kernel': 'sigmoid', 'degree': 2, 'max_iter': 2010, 'decision_function_shape': 'ovo'}. Best is trial 0 with value: 0.8736328727865764.
[I 2023-12-30 20:26:39,571] Trial 3 finished with value: 0.8666320673546475 and parameters: {'C': 3.0, 'kernel': 'poly', 'degree': 2, 'max_iter': 8010, 'decision_function_shape': 'ovo'}. Best is trial 0 with value: 0.8736328727865764.


In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 4
Best trial:
  Value: 0.8736328727865764
  Params: 
    C: 2.0
    kernel: rbf
    degree: 2
    max_iter: 2010
    decision_function_shape: ovo


In [ ]:
# История изменения от числа испытаний
optuna.visualization.plot_optimization_history(study)

In [ ]:
params = ["C",
          "kernel",
          "max_iter",
          'degree',
          'decision_function_shape'
          ]

optuna.visualization.plot_slice(study, params=params, target_name='accuracy')

# LGBM

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(model)),
                 ('clf', CatBoostClassifier(random_state=RND_STATE,
                                            # auto_class_weights='Balanced',
                                            # loss_function='MultiClass',
                                            verbose=False))])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print(classification_report(y_test, pipe.predict(X_test)))

               precision    recall  f1-score   support

         авто       0.69      0.34      0.46        58
       бизнес       0.51      0.54      0.52       277
       власть       0.51      0.42      0.46       552
        город       0.75      0.71      0.73      1182
  доброе дело       0.70      0.62      0.65        26
 недвижимость       0.00      0.00      0.00        19
  образ жизни       0.00      0.00      0.00        12
     общество       0.60      0.68      0.64      1841
     политика       0.55      0.28      0.37       134
 происшествия       0.82      0.87      0.84      1858
       разное       0.50      0.05      0.09        21
        спорт       0.90      0.90      0.90       234
строительство       0.86      0.95      0.91       216
   технологии       0.73      0.31      0.43        62
       туризм       0.50      0.11      0.18        28
      финансы       0.60      0.47      0.53       109

     accuracy                           0.70      6629
    macr

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
pipe = Pipeline([('w2v', TfidfEmbeddingVectorizer(model)),
                 ('clf', CatBoostClassifier(random_state=RND_STATE,
                                            auto_class_weights='Balanced',
                                            # loss_function='MultiClass',
                                            verbose=False))])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print(classification_report(y_test, pipe.predict(X_test)))

               precision    recall  f1-score   support

         авто       0.43      0.64      0.51        58
       бизнес       0.43      0.70      0.54       277
       власть       0.45      0.57      0.50       552
        город       0.74      0.71      0.72      1182
  доброе дело       0.45      0.77      0.57        26
 недвижимость       0.36      0.21      0.27        19
  образ жизни       0.15      0.17      0.16        12
     общество       0.69      0.48      0.56      1841
     политика       0.30      0.46      0.36       134
 происшествия       0.83      0.86      0.84      1858
       разное       0.25      0.14      0.18        21
        спорт       0.85      0.94      0.89       234
строительство       0.87      0.96      0.92       216
   технологии       0.37      0.52      0.43        62
       туризм       0.25      0.46      0.32        28
      финансы       0.50      0.63      0.56       109

     accuracy                           0.68      6629
    macr